In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nx = 10
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(nx,nx,nx),)
print(len(satoms["positions"]))
sys = pc.System()
sys.box = box
sys.atoms = satoms


2000


In [11]:
%%timeit
sys.find_neighbors(method="cutoff", 
                   cutoff=3.6)

21.9 ms ± 881 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
pcm.calculate_q(sys.atoms, 6)